In [359]:
import googlemaps
from dotenv import load_dotenv
from googlemaps import convert
import os
import h3
import pandas as pd
from typing import List, Tuple
import random
import folium

load_dotenv()
api_key = os.getenv('google_map_key')
gmaps = googlemaps.Client(key=api_key)
places = pd.read_csv('places.csv')

In [401]:
def plot_ruta(df,zoom=14):
    """
    Plot hexagons on a map based on the given dataframe.

    Parameters:
    - df (pandas.DataFrame): The dataframe containing the hexagon data. It should have the following columns:
        - lat: Latitude of the hexagon center.
        - lng: Longitude of the hexagon center.
        - h3: H3 index of the hexagon.
        - color: Color of the hexagon.

    Returns:
    - folium.Map: The map object with hexagons plotted.

    Documentation:
    - https://python-visualization.github.io/folium/quickstart.html
    
    Example usage:
    >>> plot_ruta(df)
    """

    import folium
    import h3

    # Create a map centered at the first point in the dataframe
    m = folium.Map(location=[df['lat'].iloc[0], df['lng'].iloc[0]], zoom_start=zoom)
    idx = 0

    # Add hexagons to the map
    for index, row in df.iterrows():
        # Get the vertices of the hexagon
        vertices = h3.h3_to_geo_boundary(row['h3'])
        # Convert the vertices to the format expected by folium.Polygon
        vertices = [(vertex[0], vertex[1]) for vertex in vertices]
        # Add the hexagon to the map
        folium.Polygon(locations=vertices, color=row['color'], fill=True, fill_color=row['color'], tooltip=idx).add_to(m)
        idx += 1
    zoom_square = [[df['lat'].min(),df['lng'].min()],[df['lat'].max(),df['lng'].max()]]
    m.fit_bounds(zoom_square)
    return m

In [361]:
def get_h3_index(row):
    """
    Converts latitude and longitude coordinates to H3 index.

    Parameters:
    - row: A pandas DataFrame row containing 'lat' and 'lng' columns.

    Returns:
    - H3 index corresponding to the given latitude and longitude coordinates.
    """
    return h3.geo_to_h3(row['lat'], row['lng'], 11)


In [362]:
def get_sim_route(origin:list[float] , destination:list[float],color='blue'):
    """
    Get simulated route between origin and destination using Google Maps API.

    Parameters:
    origin (list[float]): List of latitude and longitude coordinates for the origin location.
    destination (list[float]): List of latitude and longitude coordinates for the destination location.
    color (str, optional): Color of the route. Defaults to 'blue'.

    Returns:
    pandas.DataFrame: DataFrame containing latitude, longitude, h3 index, and color for each point on the route.
    """

    ruta = gmaps.directions(origin,destination)
    route_poly = ruta[0]['overview_polyline']['points']
    route_points = convert.decode_polyline(route_poly)
    route_df = pd.DataFrame(route_points, columns=['lat', 'lng']) 
    
    route_df['h3'] = route_df.apply(get_h3_index, axis=1)
    route_df = route_df.drop_duplicates(subset=['h3'])

    l=[]
    hexes = route_df['h3'].values
    for i in range(0, len(hexes)-1):
        fill = h3.h3_line(hexes[i], hexes[i+1])
        fill = [item for item in fill if item != hexes[i+1]]
        l.append(fill)

    flat_list = [item for sublist in l for item in sublist]
    l = []
    for i in range(0, len(flat_list)):
        geo= h3.h3_to_geo(flat_list[i])
        l.append([geo[0],geo[1],flat_list[i]])
    df = pd.DataFrame(l, columns=['lat', 'lng', 'h3'])
    df['color'] = color
    return df

In [402]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink', 'darkgreen', 'gray', 'black']
p1 = places.sample(1).values
p2 = places.sample(1).values
df = get_sim_route(origin=p1[0][1:3],destination=p2[0][1:3])
for i in range(0):
    p1 = places.sample(1).values
    p2 = places.sample(1).values
    df = pd.concat([df,get_sim_route(origin=p1[0][1:3],destination=p2[0][1:3],color=random.choice(colors))], ignore_index=True)

m=plot_ruta(df,zoom=10)
m

In [403]:
vertice = h3.h3_to_geo_boundary(df['h3'][1])
folium.Polygon(locations=vertice, color='red', fill=True, fill_color='red', tooltip='xx').add_to(m)
m

In [295]:
import time
def start_ruta(TreadId:int, DeviceId:int):
    p1 = places.sample(1).values
    p2 = places.sample(1).values
    df = get_sim_route(origin=p1[0][1:3],destination=p2[0][1:3])
    deviceId = DeviceId
    host = 'localhost'
    port = '3000'
    for i in range(0, len(df),30):
        print(f"treadId:{TreadId} - {i}/{len(df)}")
        time.sleep(1)
        lat = df.iloc[i].values[0]
        lng = df.iloc[i].values[1]
        print(f"http://{host}:{port}/schedule/{deviceId}/{lat}-{lng}")

In [296]:
import threading
ruta_threads = []
for i in range(3):
    t = threading.Thread(target=start_ruta, args=(i,i,))
    ruta_threads.append(t)

In [297]:
# Start the timer threads
for t in ruta_threads:
    t.start()
# Wait for all timer threads to complete
for t in ruta_threads:
    t.join()

treadId:0 - 0/42
treadId:2 - 0/172
http://localhost:3000/schedule/0/14.589545802018346--90.51951156766184
treadId:0 - 30/42
http://localhost:3000/schedule/2/14.580132775935988--90.49476432593198
treadId:2 - 30/172
http://localhost:3000/schedule/0/14.601091229271912--90.51568599736346
http://localhost:3000/schedule/2/14.577663465387275--90.48518229398107
treadId:2 - 60/172
http://localhost:3000/schedule/2/14.58745701690166--90.48893218302048
treadId:2 - 90/172
http://localhost:3000/schedule/2/14.597120832846262--90.49512192154238
treadId:2 - 120/172
http://localhost:3000/schedule/2/14.608641602115357--90.49451828936178
treadId:2 - 150/172
http://localhost:3000/schedule/2/14.60905638102246--90.48397719152273
